In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import scipy
from scipy import special, misc
from scipy.special import logsumexp
from scipy.special import digamma, gammaln, polygamma
import pandas as pd
from collections import Counter 

In [2]:
vocabulary = np.genfromtxt('ap/vocab.txt',  dtype='str')

In [3]:
def clean_up_data(corpus_file, vocabulary_file, stopwords_file):
    """
    Reads the corpus from the .txt file into a list of lists;
    a list for each document which contains a list of all the 
    words as strings.
    Input parameters:
    corpus_file - path to the corpus file
    vocabulary_file - path to the vocabulary file
    stopwords_file - path to the stopwords file
    """
    vocabulary = np.genfromtxt(vocabulary_file,  dtype='str')
    special_chars = '1234567890~!@#£$%^&*()_+,./<>?\|"]}\'[{`-'
    corpus = []
    
    # read in stopwords from file into a list
    stopwords = [] 
    with open(stopwords_file, 'r') as file:
        stop_words = file.read().replace(',', ' ')
        for word in stop_words.split():
            stopwords.append(word) 
    
    with open(corpus_file, 'r') as text:
        doc = ''
        new = False
        for line in text:
            if new: # reached a new document
                if line.strip() != '</TEXT>': # until we reach the new doc
                    for char in special_chars: # remove punctuation etc,
                        line = line.replace(char, '') 
                    doc += line
                else: # we've reached a new doc again
                    doc = doc.lower() # all words lowercase
                    words = np.array(doc.split())
                    # PETER EDIT: next two lines
                    doc = [word for word in words if (  (word not in stopwords) and (word in vocabulary)  )]
                    corpus.append(doc)
                    doc = ''
            elif line.strip() == '<TEXT>': new = True

    
    return corpus, vocabulary

In [4]:
corpus, vocabulary = clean_up_data('ap/ap.txt', 'ap/vocab.txt', 'ap/stopwords.txt')

In [5]:
def initialize_parameters(documents, vocabulary, k):
    M = len(documents)
    V = len(vocabulary)
    
    # Initialize alpha 
    # alpha = np.ones([M,k]) * 50/k # for every document, for every topic
    alpha = np.ones(k)*50/k
    eta = 5/k
    
    Lambda = np.random.rand(k,V) * 0.5 + 0.5
    
    # Initialize beta
    beta = np.zeros([k,V]) # for every topic, for every word in the vocabulary
    for i in range(k):
        beta[i] = np.random.uniform(0, 1, V)
        beta[i] = beta[i] / np.sum(beta[i])
    
    # Initialize phi and gamma
    phi = []
    gamma = np.zeros([M,k]) # for every document, for every topic
    for m in range(M):
        doc = np.array(documents[m])
        N = len(doc)
        phi.append(np.ones([N,k]) * 1/float(k)) # uniform over topics
        
        for i in range(k):
            gamma[m][i] = alpha[i] + N/float(k)
        #m += 1 # WHYYYYYYY?
        
    return alpha, eta, beta, gamma, phi, Lambda

In [6]:
def compute_lower_bound_likelihood_smoothed(M, phi, gamma, alpha, eta, document, Lambda, documents, digamma_lambda, digamma_lambda_full):
    K, V = Lambda.shape
    likelihood = np.zeros([M,1])
    N = len(document)
    
  
    digamma_gamma = digamma(gamma) - digamma(np.sum(gamma))
    
    E_theta_alpha = gammaln(alpha*K) - K * gammaln(alpha) \
                        + (alpha-1) * np.sum(digamma_gamma)
    E_z_theta = np.dot(np.sum(phi[:N,:], axis = 0), digamma_gamma)
    E_w_z_beta = np.sum(digamma_lambda * phi[:N,:])
    E_theta_gamma = gammaln(np.sum(gamma)) - np.sum(gammaln(gamma)) \
                    + np.dot(gamma - 1, digamma_gamma)
    E_z_phi = np.sum(phi[:N,:] * np.log(phi[:N,:]))

    likelihood = E_theta_alpha + E_z_theta + E_w_z_beta - E_theta_gamma - E_z_phi

    E_beta_eta = K * (gammaln(eta * V) - V * gammaln(eta)) + (eta - 1) * np.sum(digamma_lambda_full)
    E_beta_lambda = np.sum(gammaln(np.sum(Lambda, axis = 1)) - np.sum(gammaln(Lambda), axis = 1)[np.newaxis,:]) \
                    + np.sum((Lambda - 1) * digamma_lambda_full.T)

    likelihood = np.sum(likelihood) + E_beta_eta - E_beta_lambda
    
    return(likelihood)

In [7]:
def update_alpha(alpha, gamma, k, M, max_iter=50, tol=1e-4):
    
    # Maria B version
    temp = 0
    for d in range(M):
        temp_1 = np.sum(special.polygamma(0, gamma[d])) - np.sum(special.polygamma(0, np.sum(gamma, axis=1)))
    
    gradient = M * (k * special.polygamma(1, alpha) - special.polygamma(1, k*alpha))
    gradient = gradient + temp

    hessian = M * k * (k * special.polygamma(2, k*alpha) - special.polygamma(2, alpha))

    temp = gradient / (hessian * alpha + gradient + tol)
    if (alpha == 0).any():
        alpha += 0.005

    log_alpha = np.log(alpha) - temp
    alpha = np.exp(log_alpha)    
        
    return alpha

In [8]:
def update_lambda(phi, eta, doc, vocabulary, k):
    
    V = len(vocabulary)
    
    Lambda = np.ones([k, V]) * eta
    words = np.array(doc)
    for i in range(k):
        phi_ = phi[:,i]
        for j in range(V):
            word = vocabulary[j]
            indicator = np.in1d(words, word)
            indicator.astype(int)  
            Lambda[i][j] += np.dot(indicator, phi_)
                    
    return Lambda

In [9]:
def update_beta(phi, documents, vocabulary, k):
    
    M = len(documents)
    V = len(vocabulary)
    
    beta = np.zeros([k, V])
    for m, doc in enumerate(documents):
        words = np.array(doc)
        phi_m = phi[m]
        for i in range(k):
            phi_ = phi_m[:,i]
            for j in range(V):
                word = vocabulary[j]
                indicator = np.in1d(words, word)
                indicator.astype(int) 
                beta[i][j] += np.dot(indicator, phi_)
    beta = np.transpose(np.transpose(beta) / np.sum(beta, axis=1))

    return beta

In [10]:
def update_eta(eta, gamma, k, V, M, max_iter=50, tol=1e-4):

    temp = 0
    for d in range(M):
        temp_1 = np.sum(special.polygamma(0, gamma[d])) - np.sum(special.polygamma(0, np.sum(gamma, axis=1)))
    
    gradient = V * (k * special.polygamma(1, eta) - special.polygamma(1, k*eta))
    gradient = gradient + temp

    hessian = V * k * (k * special.polygamma(2, k*eta) - special.polygamma(2, eta))

    temp = gradient / (hessian * eta + gradient + tol)
    if (eta == 0):
        eta += 0.005

    log_eta = np.log(eta) - temp
    eta = np.exp(log_eta)    
        
    return eta

In [18]:
def update_phi_gamma_smoothed(M, k, phi, gamma, digamma_lambda, alpha, eta, Lambda, doc, vocabulary, documents, digamma_lambda_full, tol=1e-5, MAX_STEPS = 100):
        likelihood = 0.0
        iterations = 0
        converged = False

        N = len(doc)

        while (not converged) and (iterations < MAX_STEPS):
            iterations += 1
            
            #digamma_gamma = digamma(gamma) - digamma(np.sum(gamma))

            phi_old = phi
            phi = np.zeros([N,k])
            gamma_old = gamma

            digamma_gamma = digamma(gamma) - digamma(np.sum(gamma))
            phi[:N,:] = digamma_gamma + digamma_lambda
            phi[:N,:] = np.exp(phi[:N,:] - special.logsumexp(phi[:N,:], axis = 1)[:,np.newaxis])
            
            gamma = alpha + np.sum(phi[:N,:], axis = 0)

            # Convergence ctierion: did phi and gamma change significantly?
            if (np.linalg.norm(phi - phi_old) < tol) and (np.linalg.norm(gamma - gamma_old) < tol):              
                print(str(iterations) + ' iterations to converge.')

                likelihood += compute_lower_bound_likelihood_smoothed(M, phi, gamma, alpha, eta, doc, Lambda, documents, digamma_lambda, digamma_lambda_full)
                converged = True

        return phi, gamma, likelihood

In [12]:
def E_step_smoothed(M, phi, gamma, alpha, eta, Lambda, documents, vocabulary, k):
    print('E-step')
    
    digamma_lambda = digamma(Lambda.T) - digamma(np.sum(Lambda, axis = 1))
    for d, doc in enumerate(documents):
        phi[d], gamma[d], likelihood = update_phi_gamma_smoothed(M, k, phi[d], gamma[d], digamma_lambda[d], alpha, eta, Lambda, doc, vocabulary, Lambda, digamma_lambda)
        Lambda[d] = update_lambda(phi[d], eta, doc, vocabulary, k)
        
    return phi, gamma, Lambda, likelihood

In [13]:
def M_step_smoothed(phi, gamma, alpha, eta, documents, vocabulary, k):
    print('M-step')
    
    M = len(documents)
    V = len(vocabulary)

    alpha = update_alpha(alpha, gamma, k, M)
    eta = update_eta(eta, gamma, k, V, M)
    
    return alpha, eta

In [14]:
def variational_EM_smoothed(M, phi_init, gamma_init, alpha_init, beta_init, Lambda_init, eta_init, documents, vocabulary, k, tol=1e-5):
    print('Variational EM')
    
    M = len(documents)
    
    likelihood = 0
    likelihood_old = 0.000004
    
    iteration = 1 # Initialization step is the first step
    
    phi = phi_init
    gamma = gamma_init
    alpha = alpha_init
    beta = beta_init
    Lambda = Lambda_init
    eta = eta_init
    
    converged = False
    
    while (not converged):
        
        iteration += 1
        
        # Update parameters 
        if likelihood == 0:
            print("Likelihood==0")
            likelihood_old = 0.005
        else:
            likelihood_old = likelihood
        phi_old = phi 
        gamma_old = gamma 
        alpha_old = alpha
        beta_old = beta
        Lambda_old = Lambda
        eta_old = eta
        
    
        phi, gamma, Lambda, likelihood = \
            E_step_smoothed(M, phi_old, gamma_old, alpha_old, eta_old, Lambda_old, documents, vocabulary, k)
        alpha, eta = \
            M_step_smoothed(phi, gamma, alpha, eta, documents, vocabulary, k)
                
        if iteration > 15:
            break
        
        # check convergence
        if (np.abs((likelihood - likelihood_old) / likelihood_old) > tol):
            if (iteration > 2):
                converged = True
        
    return phi, gamma, Lambda, alpha, eta, likelihood

### Main: smoothed LDA

In [15]:
k = 3
corpus_reduced = corpus[:3]
M = len(corpus_reduced)

In [16]:
alpha_init, eta_init, beta_init, gamma_init, phi_init, Lambda_init = initialize_parameters(corpus_reduced, vocabulary, k)

In [19]:
phi, gamma, Lambda, alpha, eta, likelihood = \
variational_EM_smoothed(M, phi_init, gamma_init, alpha_init, beta_init, Lambda_init, eta_init, corpus_reduced, vocabulary, k, tol=1e-5)

Variational EM
Likelihood==0
E-step
82 iterations to converge.


ValueError: could not broadcast input array from shape (3,10473) into shape (10473)